In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2020-10-05 14:51:04--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  5.60MB/s    in 0.8s    

2020-10-05 14:51:05 (5.60 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [3]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2020-10-05 14:51:19--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  6.72MB/s    in 0.8s    

2020-10-05 14:51:21 (6.72 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


In [6]:
import torch
import torch.nn as nn #For neural network
import torch.nn.parallel #For parallel computation
import torch.optim as optim #For optimizers
import torch.utils.data 
from torch.autograd import Variable # For gradient descent

In [7]:
# We won't be using this dataset. Just for observing
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [142]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')
for i in range(2,5):
  train_set = pd.read_csv(f'ml-100k/u{i}.base', delimiter = '\t')
  training_set = np.concatenate((training_set, np.array(train_set, dtype='int')),axis=0)
  test = pd.read_csv(f'ml-100k/u{i}.test', delimiter = '\t')
  test_set = np.concatenate((test_set, np.array(test, dtype='int')),axis=0)

In [143]:
training_set.shape


(319996, 4)

In [144]:
nb_users= int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies= int(max(max(training_set[:,1]), max(test_set[:,1])))

In [145]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [146]:
training_set= torch.FloatTensor(training_set)
test_set= torch.FloatTensor(test_set)
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [147]:
#Now we build our model. Used pytorch to build this model.
class SAE(nn.Module):
  def __init__(self):
    super (SAE,self).__init__()
    self.fclayer1= nn.Linear(nb_movies,20)
    self.fclayer2= nn.Linear(20,10)
    self.fclayer3= nn.Linear(10,20)
    self.fclayer4= nn.Linear(20, nb_movies)
    self.activation= nn.Sigmoid()
  def forward(self,x):
    x= self.activation(self.fclayer1(x))
    x= self.activation(self.fclayer2(x))
    x= self.activation(self.fclayer3(x))
    x= self.fclayer4(x)
    return x


In [148]:
sae= SAE()
criterion= nn.MSELoss()
opt= optim.RMSprop(sae.parameters(), lr=0.01, weight_decay= 0.5)

In [149]:
epochs=200
for i in range(epochs):
  train_loss=0
  s=0.
  for id_user in range(nb_users):
    input= Variable(training_set[id_user]).unsqueeze(0)
    target= input.clone()
    if torch.sum(target.data>0)>0:
      output= sae(input)
      target.required_grad=False
      output[target==0]=0
      loss= criterion(output, target)
      mean_corrector= nb_movies/float(torch.sum(target.data>0)+ 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      opt.step()
      s+=1.0
  print(f'Epoch:{i}:  Loss is {train_loss/s}')





Epoch:0:  Loss is 1.5215027332305908
Epoch:1:  Loss is 1.0776312351226807
Epoch:2:  Loss is 1.047215223312378
Epoch:3:  Loss is 1.035642385482788
Epoch:4:  Loss is 1.0302492380142212
Epoch:5:  Loss is 1.0264378786087036
Epoch:6:  Loss is 1.0247740745544434
Epoch:7:  Loss is 1.0230616331100464
Epoch:8:  Loss is 1.0222581624984741
Epoch:9:  Loss is 1.0215506553649902
Epoch:10:  Loss is 1.0210528373718262
Epoch:11:  Loss is 1.0203781127929688
Epoch:12:  Loss is 1.020032525062561
Epoch:13:  Loss is 1.0199869871139526
Epoch:14:  Loss is 1.0195577144622803
Epoch:15:  Loss is 1.0192489624023438
Epoch:16:  Loss is 1.0193008184432983
Epoch:17:  Loss is 1.0190917253494263
Epoch:18:  Loss is 1.0183618068695068
Epoch:19:  Loss is 1.0172207355499268
Epoch:20:  Loss is 1.015598177909851
Epoch:21:  Loss is 1.0150617361068726
Epoch:22:  Loss is 1.0119060277938843
Epoch:23:  Loss is 1.0107215642929077
Epoch:24:  Loss is 1.0073063373565674
Epoch:25:  Loss is 1.0044883489608765
Epoch:26:  Loss is 1.00077

In [150]:
 test_loss=0
s=0.
for id_user in range(nb_users):
  input= Variable(training_set[id_user]).unsqueeze(0)
  target= Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data>0)>0:
    s+=1.
    output= sae(input)
    target.required_grad=False
    output[target==0]=0
    loss= criterion(output, target)
    mean_corrector= nb_movies/float(torch.sum(target.data>0)+ 1e-10)
    
    test_loss += np.sqrt(loss.data*mean_corrector)
print(test_loss/s)
      

tensor(0.9114)
